In [0]:
# 1. Imports and random seed
import random
from datetime import datetime, timedelta
from pyspark.sql.functions import col
random.seed(42)

In [0]:
# 2. Build 60 sample records
base_time = datetime.now() - timedelta(days=30)
records = []
for i in range(1, 61):
    order_id = i
    customer_id = f"C{random.randint(1, 15)}"
    restaurant_id = f"R{random.randint(1, 10)}"
    delivery_partner_id = f"D{random.randint(1, 8)}"
    order_value = round(random.uniform(10, 100), 2)
    delivery_time = random.randint(10, 120)  # in minutes
    order_status = random.choices(["delivered", "cancelled"], weights=[0.9, 0.1])[0]
    order_timestamp = base_time + timedelta(minutes=random.randint(0, 30*24*60))
    records.append((order_id, customer_id, restaurant_id,
                    delivery_partner_id, order_value,
                    delivery_time, order_status, order_timestamp))

In [0]:
# 3. Create DataFrame
schema = ["order_id", "customer_id", "restaurant_id",
          "delivery_partner_id", "order_value",
          "delivery_time", "order_status", "order_timestamp"]

In [0]:
df = spark.createDataFrame(records, schema)

In [0]:
df = df.withColumn("order_timestamp", col("order_timestamp").cast("timestamp"))

In [0]:
# 4. Inspect
df.show()

+--------+-----------+-------------+-------------------+-----------+-------------+------------+--------------------+
|order_id|customer_id|restaurant_id|delivery_partner_id|order_value|delivery_time|order_status|     order_timestamp|
+--------+-----------+-------------+-------------------+-----------+-------------+------------+--------------------+
|       1|         C1|           R9|                 D4|      74.44|           99|   delivered|2025-08-14 10:42:...|
|       2|         C8|          R10|                 D5|      82.85|           10|   delivered|2025-08-11 16:19:...|
|       3|        C12|           R7|                 D6|      35.01|           37|   cancelled|2025-08-19 17:35:...|
|       4|         C2|           R2|                 D7|       18.7|          118|   delivered|2025-08-16 10:51:...|
|       5|        C13|           R1|                 D8|      58.26|           58|   delivered|2025-08-17 18:09:...|
|       6|        C14|          R10|                 D6|      61

Dataset creation is completed and loaded in dataframe df



using pyspark core

1.Compute Total order value per customer


In [0]:
from pyspark.sql.functions import sum as _sum, avg

# Total order value per customer
total_value_df = (
    df.groupBy("customer_id")
      .agg(_sum("order_value").alias("total_order_value"))
      .orderBy("customer_id")
)

In [0]:
total_value_df.show()
#showing the metrics in df

+-----------+------------------+
|customer_id| total_order_value|
+-----------+------------------+
|         C1|            123.12|
|        C10|             77.32|
|        C11|            191.07|
|        C12|            281.06|
|        C13|            216.75|
|        C14|            283.01|
|        C15|402.93999999999994|
|         C2|239.45999999999998|
|         C3|             44.63|
|         C4|426.96000000000004|
|         C5|383.55000000000007|
|         C6|             24.54|
|         C7|            225.94|
|         C8|            195.55|
|         C9|            138.29|
+-----------+------------------+



2.Compute Average delivery time per restaurant

In [0]:
# Average delivery time per restaurant
avg_delivery_df = (
    df.groupBy("restaurant_id")
      .agg(avg("delivery_time").alias("avg_delivery_time_minutes"))
      .orderBy("restaurant_id")
)

In [0]:
avg_delivery_df.show()
#showing the metrics in df

+-------------+-------------------------+
|restaurant_id|avg_delivery_time_minutes|
+-------------+-------------------------+
|           R1|                     64.1|
|          R10|                     56.5|
|           R2|                     77.8|
|           R3|        92.33333333333333|
|           R4|                     66.0|
|           R5|                    75.25|
|           R6|                     69.0|
|           R7|                     46.2|
|           R8|                    67.75|
|           R9|        74.16666666666667|
+-------------+-------------------------+



Using Pyspark SQL

In [0]:
# Register temp view
df.createOrReplaceTempView("orders")

1.Identify Customers with frequent cancellations

In [0]:
%sql
-- 1. Frequent cancellations
SELECT
  customer_id,
  COUNT(*) AS cancellation_count
FROM orders
WHERE order_status = 'cancelled'
GROUP BY customer_id
HAVING COUNT(*) > 1
ORDER BY cancellation_count DESC;

customer_id,cancellation_count
C7,2


Databricks visualization. Run in Databricks to view.

2. Calculate peak order hours using group by hour(timestamp)

In [0]:
%sql
-- 2. Peak order hours
SELECT
  HOUR(order_timestamp) AS hour_of_day,
  COUNT(*) AS orders_count
FROM orders
GROUP BY HOUR(order_timestamp)
ORDER BY orders_count DESC;

hour_of_day,orders_count
18,5
23,5
12,5
17,4
21,4
13,4
16,3
10,3
8,3
5,3


Databricks visualization. Run in Databricks to view.

Done with showing above metrics in dataframe 

In [0]:
#Done with showing above metrics in dataframe 

In [0]:
display(total_value_df)


customer_id,total_order_value
C1,123.12
C10,77.32
C11,191.07
C12,281.06
C13,216.75
C14,283.01
C15,402.93999999999994
C2,239.45999999999998
C3,44.63
C4,426.96000000000004


Databricks visualization. Run in Databricks to view.

In [0]:
display(avg_delivery_df)

restaurant_id,avg_delivery_time_minutes
R1,64.1
R10,56.5
R2,77.8
R3,92.33333333333333
R4,66.0
R5,75.25
R6,69.0
R7,46.2
R8,67.75
R9,74.16666666666667


Databricks visualization. Run in Databricks to view.

In [0]:
display(df)
#this is my original dataset which I used for my analysis

order_id,customer_id,restaurant_id,delivery_partner_id,order_value,delivery_time,order_status,order_timestamp
1,C1,R9,D4,74.44,99,delivered,2025-08-14T10:42:18.741Z
2,C8,R10,D5,82.85,10,delivered,2025-08-11T16:19:18.741Z
3,C12,R7,D6,35.01,37,cancelled,2025-08-19T17:35:18.741Z
4,C2,R2,D7,18.7,118,delivered,2025-08-16T10:51:18.741Z
5,C13,R1,D8,58.26,58,delivered,2025-08-17T18:09:18.741Z
6,C14,R10,D6,61.96,100,delivered,2025-08-14T18:51:18.741Z
7,C13,R5,D2,86.98,120,delivered,2025-08-17T01:33:18.741Z
8,C8,R6,D3,43.32,36,delivered,2025-09-02T21:45:18.741Z
9,C2,R10,D3,58.07,41,delivered,2025-08-21T16:23:18.741Z
10,C5,R9,D4,71.62,117,delivered,2025-08-06T23:01:18.741Z
